# Extracting Text from PDFs

## Import relevant libraries 

In [22]:
# -First, import the PyPDF2 module
# -Then open "xxxxxx".pdf in read binary mode and store it in a pdffileObj
# -To get a PdfFileReader object that represents this PDF, call PyPDF2.PdfFileReader()
import PyPDF2
import textract
import nltk
import pandas as pd
import numpy as np
import re
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load and Read PDF

In [23]:
pdfFileObj = open('meetingminutes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

## Create A loop  

In [24]:
#Discerning the number of pages will allow us to parse through all the pages.
num_pages = pdfReader.numPages
count = 0
text = ""

#The while loop will read each page.
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
if text != "":
    text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text.
else:
    text = textract.process(fileurl, method='tesseract', language='eng')
#Now we have a text variable that contains all the text derived from our PDF file. Type print(text) to see what it contains. It likely contains a lot of spaces, possibly junk such as '\n,' etc.
#Now, we will clean our text variable and return it as a list of keywords.

## Clean Text - Convert to Keywords 

In [25]:
#The word_tokenize() function will break our text phrases into individual words.
tokens = word_tokenize(text)
#We'll create a new list that contains punctuation we wish to clean.
punctuations = ['(',')',';',':','[',']',',']
#We initialize the stopwords variable, which is a list of words like "The," "I," "and," etc. that don't hold much value as keywords.
stop_words = stopwords.words('english')
#We create a list comprehension that only returns a list of words that are NOT IN stop_words and NOT IN punctuations.
keywords = [word for word in tokens if not word in stop_words and not word in punctuations]

In [26]:
print(text)

7PATTERN MATCHING WITH 
REGULAR EXPRESSIONS
You may be familiar with searching for text 
by pressing 
CTRL
-F and entering the words 
you™re looking for. 
Regular expressions
 go one 
step further: they allow you to specify a 
pattern
 of 
text to search for. You may not know a business™s exact 
phone number, but if you live in the United States or
Canada, you know it will be three digits, followed by a hyphen, and then 
four more digits (and optionally, a three-digit area code at the start). This 
is how you, as a human, know a phone number when you see it: 415-555-
1234 is a phone number, but 4,155,551,234 is not. 
We also recognize all sorts of other text patterns every day: email 
addresses have @ symbols in the middle, US social security numbers have 
nine digits and two hyphens, website URLs often have periods and forward 
slashes, news headlines use title case, social media hashtags begin with # 
and contain no spaces, and more.
162
   Chapter 7
Regular expressions are helpful, 

In [27]:
len(text)

53877

## Extract keywords 

In [28]:
keywords = re.findall(r'[a-zA-Z]\w+',text)
len(keywords)#Total keywords in document

7866

In [30]:
#Dataframe with unique keywords to avoid repetition in rows
df = pd.DataFrame(list(set(keywords)),columns=['keywords']) 


In [31]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf

In [32]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [33]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)


,keywords,number_of_times_word_appeared,tf,idf,tf_idf
831,re,764,0.014180,-6.638568,-0.094138
1206,in,730,0.013549,-6.593045,-0.089332
298,the,641,0.011897,-6.463029,-0.076894
989,at,626,0.011619,-6.439350,-0.074819
266,es,486,0.009021,-6.186209,-0.055803
548,on,482,0.008946,-6.177944,-0.055270
84,an,417,0.007740,-6.033086,-0.046695
1248,ing,348,0.006459,-5.852202,-0.037800
181,or,334,0.006199,-5.811141,-0.036025
198,nd,285,0.005290,-5.652489,-0.029901


In [34]:
print(text.find("return"))

2582


In [63]:
for i in range(0,10):
    print(keywords[i])


PATTERN
MATCHING
WITH
REGULAR
EXPRESSIONS
You
may
be
familiar
with
